In [5]:
import cv2
import glob
import re
from PIL import Image
from pytesseract import pytesseract
from matplotlib import pyplot as plt




In [6]:
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


In [77]:
path = "Findmind_images/*.*"
count = 1
for file in glob.glob(path):
    image_read = cv2.imread(file)
    osd = pytesseract.image_to_osd(image_read)
    angle = re.search('(?<=Rotate: )\d+', osd).group(0)
   
    if int(angle) == 90:
        (h, w) = image_read.shape[:2]
        # calculate the center of the image
        center = (w / 2, h / 2)

        scale = 1.0

        # Perform the counter clockwise rotation holding at the center
        # 90 degrees
        M = cv2.getRotationMatrix2D(center, -int(angle), scale)
        rotated = cv2.warpAffine(image_read, M, (2*w, 2*h))
        img_resize = cv2.resize(rotated, None, fx = 2, fy = 2, interpolation = cv2.INTER_LINEAR)
        gray = get_grayscale(img_resize)
        image = gray
        image1 = remove_noise(image)
        text = pytesseract.image_to_string(image1)
        
        match = re.search(r'(\d+/\d+/\d+)',text)
        print("PANCARD:",count)
        print("DOB:",match.group(1))    
        match1 =  re.search(r'([A-Z].*\d+.[A-Z])', text)
        X = match1.group(1)
        print("NUMBER:",re.sub('[^A-Z 0-9]','',X))
        count = count+1
        
    elif int(angle) ==0:
        img_resize = cv2.resize(image_read, None, fx = 2, fy = 2, interpolation = cv2.INTER_LINEAR)
        gray = get_grayscale(img_resize)
        image = gray
        image1 = remove_noise(image)
        text = pytesseract.image_to_string(image1)
        match = re.search(r'(\d+/\d+/\d+)',text)
        print("PANCARD:",count)
        print("DOB:",match.group(1))
        match1 =  re.search(r'([A-Z]*\d+[A-Z])', text)
        Y = match1.group(1)
        print("NUMBER:",re.sub('[^A-Z 0-9]','',Y))
        count = count+1
        
        

PANCARD: 1
DOB: 18/03/1996
NUMBER: BGYPJO129A
PANCARD: 2
DOB: 30/01/1997
NUMBER: ELWPM8089J
PANCARD: 3
DOB: 13/09/1996
NUMBER: BXAPC1000L
